
requirment:
python==3.6.5
flask

In [4]:
import importlib
import featurebinning as fb
importlib.reload(fb)

import pandas as pd

In [5]:
data1 = pd.read_pickle("./data_sample.pkl")
display(data1.shape)
display(data1.head())
wt = fb.WideTable(data1)
wt.run_app()

(20000, 50)

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var40,var41,var42,var43,var44,var45,var46,var47,var48,target
188489,1.5,1.5,600.0,3.0,1.0,2.2,2.666667,4.000000,6.0,200.0,...,2.3061,0.0,2.000000,0.0,5.5,500.0,3.6,2.5,6000.0,0
133130,0.5,1.5,200.0,NaN,1.0,NaN,50.000000,0.000000,16.0,-99998.0,...,69.1830,-10000000.0,1.111111,0.0,22.5,100.0,NaN,0.5,8000.0,0
118749,0.5,1.5,600.0,NaN,1.0,NaN,8.000000,NaN,5.0,-99998.0,...,6.9183,-10000000.0,1.636364,0.0,5.5,100.0,NaN,0.5,4000.0,0
4345,0.5,0.5,5.0,NaN,1.0,NaN,0.000000,NaN,0.0,-99998.0,...,0.0000,-10000000.0,0.666667,0.0,1.5,100.0,NaN,0.5,4000.0,0
132270,0.5,1.5,200.0,0.0,1.0,3.0,60.000000,2.857143,12.0,200.0,...,70.0000,0.0,1.515152,0.0,16.5,100.0,0.0,0.5,8000.0,0


running
 * Serving Flask app "binning_server" (lazy loading)
 * Environment: development
 * Debug mode: off


 * Running on http://127.0.0.1:50000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Oct/2023 09:53:38] "GET /test?var=var0 HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2023 09:53:38] "GET /static/range-slider.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Oct/2023 09:53:38] "GET /static/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Oct/2023 09:53:38] "GET /static/echarts.min.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Oct/2023 09:53:38] "GET /static/range-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Oct/2023 09:53:46] "GET /shutdown HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2023 09:53:59] "GET /test?var=var0 HTTP/1.1" 200 -


In [14]:
%%HTML
<iframe src="http://127.0.0.1:50000/test?var=var7&step=1" width="90%" height="720px" style="background-color:antiquewhite"></iframe>

In [15]:
%%HTML
<iframe src="http://127.0.0.1:50000/shutdown"  style="background-color:antiquewhite"></iframe>

In [12]:
#TODO:
# 1) url param加min和max
# 2） hover 在点上的时候，对应bar高亮

In [8]:
from flask import jsonify
import json
json.dumps(pd.DataFrame([[np.inf,np.nan,-np.inf,1,"a"]]).to_dict(orient="records"  ))

'[{"0": Infinity, "1": NaN, "2": -Infinity, "3": 1, "4": "a"}]'